In [7]:
import torch
import numpy as np
from ultralytics import YOLO
from torchmetrics.detection import IntersectionOverUnion
import os

In [2]:
model_detection = YOLO('runs/detect/train10/weights/best.pt')

In [3]:
results_detection = model_detection.predict(source = 'test_new/images', save_txt = True)


image 1/5 /blue/eel5840/b.gandhi/Final Project/Training Data/test_new/images/176.jpg: 640x640 1 1, 1 6, 1 7, 7.3ms
image 2/5 /blue/eel5840/b.gandhi/Final Project/Training Data/test_new/images/38.jpg: 640x640 1 3, 1 8, 8.6ms
image 3/5 /blue/eel5840/b.gandhi/Final Project/Training Data/test_new/images/438.jpg: 640x640 1 3, 1 4, 1 8, 6.9ms
image 4/5 /blue/eel5840/b.gandhi/Final Project/Training Data/test_new/images/7.jpg: 640x640 1 7, 6.8ms
image 5/5 /blue/eel5840/b.gandhi/Final Project/Training Data/test_new/images/96.jpg: 640x640 2 6s, 1 9, 6.8ms
Speed: 2.1ms preprocess, 7.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict19
5 labels saved to runs/detect/predict19/labels


In [4]:
result_dir = results_detection[0].save_dir

In [6]:
import os
import torch
from torchmetrics.detection.iou import IntersectionOverUnion

def yolo_to_abs_coords(yolo_data, img_width, img_height):
    """
    Convert YOLO (class, x_center, y_center, width, height) to (xmin, ymin, xmax, ymax).
    """
    abs_boxes = []
    labels = []
    for entry in yolo_data:
        cls, x_center, y_center, width, height = entry
        xmin = (x_center - width / 2) * img_width
        ymin = (y_center - height / 2) * img_height
        xmax = (x_center + width / 2) * img_width
        ymax = (y_center + height / 2) * img_height
        abs_boxes.append([xmin, ymin, xmax, ymax])
        labels.append(int(cls))
    return torch.tensor(abs_boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

def parse_yolo_file(file_path):
    """
    Parse a YOLO .txt file into a list of tuples (class, x_center, y_center, width, height).
    """
    with open(file_path, "r") as f:
        lines = f.readlines()
    return [list(map(float, line.strip().split())) for line in lines]

def prepare_data(file_paths, img_width, img_height):
    """
    Prepare data for multiple YOLO files into the required format for PyTorch metrics.
    """
    data = []
    for file_path in file_paths:
        yolo_data = parse_yolo_file(file_path)
        boxes, labels = yolo_to_abs_coords(yolo_data, img_width, img_height)
        data.append({"boxes": boxes, "labels": labels})
    return data

img_width = 640
img_height = 640

# Directories containing YOLO label files
preds_dir = result_dir + "/labels"
targets_dir = "test_new/labels"

# Collect all .txt files in directories
preds_files = [os.path.join(preds_dir, f) for f in os.listdir(preds_dir) if f.endswith(".txt")]
targets_files = [os.path.join(targets_dir, f) for f in os.listdir(targets_dir) if f.endswith(".txt")]

# Ensure the order of files matches (or sort them)
preds_files.sort()
targets_files.sort()

# Prepare prediction and target data
preds = prepare_data(preds_files, img_width, img_height)
targets = prepare_data(targets_files, img_width, img_height)

# Compute IoU using PyTorch metrics
iou_metric = IntersectionOverUnion()
iou_score = iou_metric(preds, targets)
print(f"IoU Score: {iou_score}")

IoU Score: {'iou': tensor(0.7241)}
